# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [1]:
import pandas as pd
import boto3
import sagemaker
import os

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [3]:
# should be the name of directory you created to save your features data
data_dir = 'plagiarism_data/'

# set prefix, a descriptive name for a directory  
prefix = 'sagemaker/plagiarism'

# upload all data to S3
test_location = sagemaker_session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
train_location = sagemaker_session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

In [4]:
test = pd.read_csv(os.path.join(data_dir, 'test.csv'), header=None)
test_no_label = test.iloc[:, 1:]
test_no_label.to_csv(os.path.join(data_dir, 'test_no_label.csv'), header=False, index=False)

test_no_label_location = sagemaker_session.upload_data(os.path.join(data_dir, 'test_no_label.csv'), key_prefix=prefix)

### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

boston-xgboost-HL/output/xgboost-2020-05-22-13-42-04-930/output/model.tar.gz
boston-xgboost-HL/test.csv
boston-xgboost-HL/train.csv
boston-xgboost-HL/validation.csv
boston-xgboost-LL/batch-bransform/test.csv.out
boston-xgboost-LL/output/boston-xgboost-2020-05-22-14-05-58/output/model.tar.gz
boston-xgboost-LL/test.csv
boston-xgboost-LL/train.csv
boston-xgboost-LL/validation.csv
boston-xgboost-deploy-hl/output/xgboost-2020-05-22-18-46-48-536/output/model.tar.gz
boston-xgboost-deploy-hl/train.csv
boston-xgboost-deploy-hl/validation.csv
sagemaker-pytorch-2020-05-25-01-55-44-345/debug-output/training_job_end.ts
sagemaker-pytorch-2020-05-25-01-55-44-345/output/model.tar.gz
sagemaker-pytorch-2020-05-25-01-55-44-345/source/sourcedir.tar.gz
sagemaker-pytorch-2020-05-25-03-00-42-485/sourcedir.tar.gz
sagemaker-pytorch-2020-05-25-03-16-40-271/sourcedir.tar.gz
sagemaker-pytorch-2020-05-25-03-45-55-027/sourcedir.tar.gz
sagemaker-pytorch-2020-05-25-03-59-23-026/sourcedir.tar.gz
sagemaker-pytorch-2020

---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

In [6]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_pytorch/train.py

import argparse
import json
import os
import pandas as pd
import torch
import torch.optim as optim
import torch.utils.data

# imports the model in model.py by name
from model import BinaryClassifier

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BinaryClassifier(model_info['input_features'], model_info['hidden_dim'], model_info['output_dim'])

    # Load the stored model parameters.
    model_path = os.path.join(model_dir, 'model.pth')
    with open(model_path, 'rb') as f:
        model.load_state_dict(torch.load(

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

In [7]:
from sagemaker import get_execution_role

role = get_execution_role()

In [8]:
# your import and estimator code, here
from sagemaker.pytorch import PyTorch

twoLayerNN = PyTorch(entry_point="train.py",
                    source_dir="source_pytorch",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    hyperparameters={
                        'epochs': 500,
                    })

## Build in XGBoost estimator

In [9]:
from sagemaker.amazon.amazon_estimator import get_image_uri


container = get_image_uri(sagemaker_session.boto_region_name, 'xgboost')

# Create XGBoost estimator
xgb = sagemaker.estimator.Estimator(container, # The location of the container we wish to use
                                    role,                                    # What is our current IAM Role
                                    train_instance_count=1,                  # How many compute instances
                                    train_instance_type='ml.m4.xlarge',      # What kind of compute instances
                                    output_path=f's3://{bucket}/{prefix}/output',
                                    sagemaker_session=sagemaker_session)

# Set hyperparameters
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        early_stopping_rounds=10,
                        num_round=500)

	get_image_uri(region, 'xgboost', '0.90-1').


## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [10]:
%%time

# Train your estimator on S3 training data
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')
xgb.fit({'train': s3_input_train})

2020-06-03 19:20:54 Starting - Starting the training job...
2020-06-03 19:20:56 Starting - Launching requested ML instances.........
2020-06-03 19:22:28 Starting - Preparing the instances for training......
2020-06-03 19:23:44 Downloading - Downloading input data...
2020-06-03 19:24:18 Training - Downloading the training image..
2020-06-03 19:24:49 Uploading - Uploading generated training model
2020-06-03 19:24:49 Completed - Training job completed
Arguments: train
[2020-06-03:19:24:38:INFO] Running standalone xgboost training.
[2020-06-03:19:24:38:INFO] Path /opt/ml/input/data/validation does not exist!
[2020-06-03:19:24:38:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 8502.23mb
[2020-06-03:19:24:38:INFO] Determined delimiter of CSV input is ','
[19:24:38] S3DistributionType set as FullyReplicated
[19:24:38] 70x3 matrix with 210 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[19:24:38] src/tree/updater_p

In [11]:
%%time 
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')
twoLayerNN.fit({'train': s3_input_train})

2020-06-03 19:25:06 Starting - Starting the training job...
2020-06-03 19:25:08 Starting - Launching requested ML instances......
2020-06-03 19:26:12 Starting - Preparing the instances for training......
2020-06-03 19:27:27 Downloading - Downloading input data...
2020-06-03 19:27:56 Training - Downloading the training image...
2020-06-03 19:28:34 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-06-03 19:28:34,896 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-06-03 19:28:34,919 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-06-03 19:28:37,951 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-06-03 19:28:38,220 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-06-03 19:28:38,220 sagemak

Training seconds: 99
Billable seconds: 99
CPU times: user 755 ms, sys: 63.3 ms, total: 819 ms
Wall time: 4min 12s


## EXERCISE: Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

In [12]:
%%time

# deploy your model to create a predictor
predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-------------!CPU times: user 250 ms, sys: 5.33 ms, total: 255 ms
Wall time: 6min 31s


In [13]:
%%time

predictorPyTorch = twoLayerNN.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-------------!CPU times: user 237 ms, sys: 25.7 ms, total: 263 ms
Wall time: 6min 31s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [14]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

### Just add the code for batch transform

In [15]:
xgb_transformer = xgb.transformer(instance_count = 1, instance_type = 'ml.m4.xlarge')

In [16]:
xgb_transformer.transform(test_no_label_location, content_type='text/csv', split_type='Line')

In [17]:
xgb_transformer.wait()

....................Arguments: serve
[2020-06-03 19:45:28 +0000] [1] [INFO] Starting gunicorn 19.7.1
[2020-06-03 19:45:28 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2020-06-03 19:45:28 +0000] [1] [INFO] Using worker: gevent
[2020-06-03 19:45:28 +0000] [38] [INFO] Booting worker with pid: 38
[2020-06-03 19:45:28 +0000] [39] [INFO] Booting worker with pid: 39
[2020-06-03 19:45:28 +0000] [40] [INFO] Booting worker with pid: 40
[2020-06-03 19:45:28 +0000] [41] [INFO] Booting worker with pid: 41
[2020-06-03:19:45:28:INFO] Model loaded successfully for worker : 38
[2020-06-03:19:45:28:INFO] Model loaded successfully for worker : 39
[2020-06-03:19:45:28:INFO] Model loaded successfully for worker : 40
[2020-06-03:19:45:28:INFO] Model loaded successfully for worker : 41
[2020-06-03:19:45:56:INFO] Sniff delimiter as ','
[2020-06-03:19:45:56:INFO] Determined delimiter of CSV input is ','
2020-06-03T19:45:56.478:[sagemaker logs]: MaxConcurrentTransforms=4, MaxPayloadInMB=6, BatchStra

In [18]:
!aws s3 cp --recursive $xgb_transformer.output_path $data_dir

download: s3://sagemaker-us-east-2-505506697994/xgboost-2020-06-03-19-42-22-485/test_no_label.csv.out to plagiarism_data/test_no_label.csv.out


In [19]:
predictions = pd.read_csv(os.path.join(data_dir, 'test_no_label.csv.out'), header=None)
predictions = [round(num) for num in predictions.squeeze().values]

In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.96

In [21]:
# First: generate predicted, class labels
# Build-in XGBoost model
from sagemaker.predictor import csv_serializer
import numpy as np

predictor.content_type = 'text/csv'
predictor.serializer = csv_serializer
test_y_preds = predictor.predict(test_x.values).decode('utf-8')
test_y_preds = np.fromstring(test_y_preds, sep=',')
test_y_preds[test_y_preds > 0.5] = 1
test_y_preds[test_y_preds <= 0.5] = 0

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [22]:
# Two layer NN with PyTorch
import torch
import torch.utils.data

test_y_predsPyTorch = predictorPyTorch.predict(torch.from_numpy(test_x.values).float().squeeze()).squeeze()
print(test_y_predsPyTorch)
# test_y_predsPyTorch = np.fromstring(test_y_predsPyTorch, sep=',')
test_y_predsPyTorch[test_y_predsPyTorch > 0.5] = 1
test_y_predsPyTorch[test_y_predsPyTorch <= 0.5] = 0

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_predsPyTorch)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

[1.         0.9999957  0.9969976  0.7841566  0.999982   1.
 0.32251078 0.20215987 0.25545782 0.3406029  0.26459068 0.3247251
 0.57095927 0.9999999  1.         0.95297366 0.99544454 1.
 0.20215987 1.         0.36910212 1.         1.         0.20215987
 0.2794794 ]
Test passed!


## EXERCISE: Determine the accuracy of your model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [23]:
# Second: calculate the test accuracy
tp = np.logical_and(test_y, test_y_preds).sum()
fp = np.logical_and(1-test_y, test_y_preds).sum()
tn = np.logical_and(1-test_y, 1-test_y_preds).sum()
fn = np.logical_and(test_y, 1-test_y_preds).sum()

# calculate binary classification metrics
recall = tp / (tp + fn)
precision = tp / (tp + fp)
accuracy = (tp + tn) / (tp + fp + tn + fn)

print(pd.crosstab(test_y, test_y_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
print("\n{:<11} {:.3f}".format('Recall:', recall))
print("{:<11} {:.3f}".format('Precision:', precision))
print("{:<11} {:.3f}".format('Accuracy:', accuracy))
print()


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

prediction (col)  0.0  1.0
actual (row)              
0                   9    1
1                   0   15

Recall:     1.000
Precision:  0.938
Accuracy:   0.960


Predicted class labels: 
[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 0.]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


In [24]:
# Second: calculate the test accuracy (Pytorch)
tp = np.logical_and(test_y, test_y_predsPyTorch).sum()
fp = np.logical_and(1-test_y, test_y_predsPyTorch).sum()
tn = np.logical_and(1-test_y, 1-test_y_predsPyTorch).sum()
fn = np.logical_and(test_y, 1-test_y_predsPyTorch).sum()

# calculate binary classification metrics
recall = tp / (tp + fn)
precision = tp / (tp + fp)
accuracy = (tp + tn) / (tp + fp + tn + fn)

print(pd.crosstab(test_y, test_y_predsPyTorch, rownames=['actual (row)'], colnames=['prediction (col)']))
print("\n{:<11} {:.3f}".format('Recall:', recall))
print("{:<11} {:.3f}".format('Precision:', precision))
print("{:<11} {:.3f}".format('Accuracy:', accuracy))
print()


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_predsPyTorch)
print('\nTrue class labels: ')
print(test_y.values)

prediction (col)  0.0  1.0
actual (row)              
0                  10    0
1                   0   15

Recall:     1.000
Precision:  1.000
Accuracy:   1.000


Predicted class labels: 
[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 0.]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

** Answer**: 
> 1 false positive for XGBoost. It might bc there are more positive samples in training. 

### Question 2: How did you decide on the type of model to use? 

** Answer**:
> I want to try the state-of-the-art gradient boost algorithm together with a simple two-layer neural network.

----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [25]:
# uncomment and fill in the line below!
# <name_of_deployed_predictor>.delete_endpoint()
xgb.delete_endpoint()
twoLayerNN.delete_endpoint()

### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [26]:
# deleting bucket, uncomment lines below

# bucket_to_delete = boto3.resource('s3').Bucket(bucket)
# bucket_to_delete.objects.all().delete()

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!